In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch import nn, optim
from torch.nn import functional as F
import pytorch_lightning as pl
from torchmetrics.functional import accuracy

import pandas as pd
import numpy as np
import os
from IPython.display import clear_output

try:
    from country_region import *
except Exception as e:
    print(e)

In [3]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

fix_all_seeds(42)

## Load data

In [4]:
meta = pd.read_csv('4000_PCS_human_origins/v44.3_HO_public.anno', sep='\t')
pcs = pd.read_csv('4000_PCS_human_origins/pcs.txt', sep='\t')

In [5]:
meta

,Index,Version ID,Master ID,Publication (or OK to use in a paper),Representative contact,"Date mean in BP [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]","Full Date: One of two formats. (Format 1) 95.4% CI calibrated radiocarbon age (Conventional Radiocarbon Age BP, Lab number) e.g. 2624-2350 calBCE (3990±40 BP, Ua-35016). (Format 2) Archaeological context range, e.g. 2500-1700 BCE",Group Label,Locality,Country,Lat.,Long.,Data source,Coverage on autosomal targets,SNPs hit on autosomal targets,Sex,"Library type (minus=no.damage.correction, half=damage.retained.at.last.position, plus=damage.fully.corrected, ds=double.stranded.library.preparation, ss=single.stranded.library.preparation)","ASSESSMENT (Xcontam interval is listed if lower bound is >0.005, ""QUESTIONABLE"" if lower bound is 0.01-0.02, ""QUESTIONABLE_CRITICAL"" or ""FAIL"" if lower bound is >0.02) (mtcontam confidence interval is listed if coverage >2 and upper bound is <0.98: 0.9-0.95 is ""QUESTIONABLE""; <0.9 is ""QUESTIONABLE_CRITICAL"", questionable status gets overriden by ANGSD with PASS if upper bound of contamination is <0.01 and QUESTIONABLE if upper bound is 0.01-0.05) (damage for ds.half is ""QUESTIONABLE_CRITICAL/FAIL"" if <0.01, ""QUESTIONABLE"" for 0.01-0.03, and recorded but passed if 0.03-0.05; libraries with fully-treated last base are ""QUESTIONABLE_CRITICAL"" or ""FAIL"" if <0.03, ""QUESTIONABLE"" if 0.03-0.06, and recorded but passed if 0.06-0.1) (sexratio is QUESTIONABLE if [0.03,0.10] or [0.30,0.35); QUESTIONABLE_CRITICAL/FAIL if (0.10,0.30))"
0,1798,MAL-005,MAL-005,SkoglundCell2017,Garrett Hellenthal / Saioa Lopez / Mark Thomas...,0,..,Malawi_Yao,Dedza // Yao,Malawi,-14.166667,34.33333,Fall2015,..,585645,M,..,PASS (genotyping)
1,1799,MAL-009,MAL-009,SkoglundCell2017,Garrett Hellenthal / Saioa Lopez / Mark Thomas...,0,..,Malawi_Yao,Machinga // Yao,Malawi,-14.862605,35.574122,Fall2015,..,582189,M,..,PASS (genotyping)
2,1800,MAL-011,MAL-011,SkoglundCell2017,Garrett Hellenthal / Saioa Lopez / Mark Thomas...,0,..,Malawi_Chewa,Mchinga // Chichewa,Malawi,-14.862605,35.574122,Fall2015,..,579844,M,..,PASS (genotyping)
3,1801,MAL-012,MAL-012,SkoglundCell2017,Garrett Hellenthal / Saioa Lopez / Mark Thomas...,0,..,Malawi_Chewa,Salima // Chichewa,Malawi,-13.75,34.5,Fall2015,..,585204,M,..,PASS (genotyping)
4,1802,MAL-014,MAL-014,SkoglundCell2017,Garrett Hellenthal / Saioa Lopez / Mark Thomas...,0,..,Malawi_Chewa,Nambuma // Chichewa,Malawi,-13.703473,33.597743,Fall2015,..,584410,M,..,PASS (genotyping)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13192,31953,VK94.SG,VK94,MargaryanWillerslevNature2020,"Nielsen, Rasmus; Werge, Thomas; Willerslev, Eske",1000,800-1100 CE,Denmark_Viking.SG,"Sealand, Gl. Lejre",Denmark,55.62,11.97,Shotgun,0.14,63365,F,ds.minus,PASS (literature)
13193,31954,VK95.SG,VK95,MargaryanWillerslevNature2020,"Nielsen, Rasmus; Werge, Thomas; Willerslev, Eske",850,900-1300 CE,Iceland_Viking.SG,Hofstadir,Iceland,65.61,-17.16,Shotgun,1.32,422417,M,ds.minus,PASS (literature)
13194,31955,VK98.SG,VK98,MargaryanWillerslevNature2020,"Nielsen, Rasmus; Werge, Thomas; Willerslev, Eske",850,900-1300 CE,Iceland_Viking.SG,Hofstadir,Iceland,65.61,-17.16,Shotgun,2.49,524046,M,ds.minus,PASS (literature)
13195,31956,VK99.SG,VK99,MargaryanWillerslevNature2020,"Nielsen, Rasmus; Werge, Thomas; Willerslev, Eske",850,900-1300 CE,Iceland_Viking.SG,Hofstadir,Iceland,65.61,-17.16,Shotgun,0.74,320753,F,ds.minus,PASS (literature)


## Prepare data


In [4]:
from typing import Union, Tuple

def prepare_data(pcs: pd.DataFrame, meta: pd.DataFrame, num_of_pcs: Union[None, int]=None, target: str='region') -> Tuple[torch.Tensor, torch.Tensor, dict]:
    """
    pcs: The principal components of the data
    meta: The meta data of the datam, as given by https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/index_v44.3.html
    num_of_pcs: The number of principal components to include from the pcs dataframe. If None, all pcs are included.

    Output:
        x: is the features in a torch Tensor
        y: is the targets in a torch Tensor
        idx_to_country: a dictionary that maps the index in the target vector, to the representative country
    """
    
    filter_meta = meta[['Version ID', 'Country']]

    merged = pd.merge(filter_meta, pcs, how='left', left_on=['Version ID'], right_on=['IID'])
    merged = merged[merged['Country'] != '..'].reset_index(drop=True) # first country is '..', so we remove that
    
    
    if target.lower() == 'country':
        countries = merged['Country'].unique()
        idx_to_country = dict(zip(countries, np.arange(len(countries))))
        merged['Country'] = merged.apply(lambda row: idx_to_country[row['Country']], axis=1)
        # one_hot_countries = pd.get_dummies(merged['Country'])  
    
    elif target.lower() == 'region':
        merged['Region'] = merged['Country'].map(country_region)
        regions = merged['Region'].unique()
        idx_to_region = dict(zip(regions, np.arange(len(regions))))
        merged['Region'] = merged.apply(lambda row: idx_to_country[row['Region']], axis=1)
    
    
    # y = torch.from_numpy(one_hot_countries.values)
    y = torch.from_numpy(merged['Country'].values)
    x_all = merged.iloc[:, 4:].values

    if num_of_pcs is None:
        x = x_all
    else:
        x = x_all[:, :num_of_pcs]
    
    x = torch.from_numpy(x)
    
    # countries = one_hot_countries.columns
    # idx_to_country = dict(zip(np.arange(len(countries)), countries))

    return x.float(), y.long(), idx_to_country
    

In [5]:
def create_dataloaders(x: torch.Tensor, y: torch.Tensor, batch_size: int = 64, train_shuffle: bool = False, save: bool = False):
    """
    x: Features which were created in the prepare_data function
    y: Targets which were created in the prepare_data function
    batch_size: Chooses the batch size of the dataloader
    save: If the dataloaders should be saved

    Output: Two dataloaders, one for training and another for validation
    """
    
    # First the full dataset is created from the tensors
    dataset = TensorDataset(x, y)

    # Then using random_split a test and val set is created
    train_size = int(0.8*len(dataset))
    test_size = len(dataset) - train_size
    train_set, val_set = random_split(dataset, [train_size, test_size])

    # Then the dataloaders can initialized
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=train_shuffle)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    if save:
        torch.save(train_loader, 'processed_data/train_loader.pth')
        torch.save(val_loader, 'processed_data/val_loader.pth')

    return train_loader, val_loader

In [6]:
x, y, idx_to_country = prepare_data(pcs, meta, num_of_pcs=4000)

In [7]:
train_loader, val_loader = create_dataloaders(x, y, batch_size=64, train_shuffle=True)

## Models

In [8]:
# Simple ffnn
class ffnn(pl.LightningModule):
    def __init__(self, input_size: int=100, layer_1: int=2000, layer_2: int=1500, layer_3: int=1000, layer_4: int=500,
                 dropout: float=0.25,
                 lr: float=1.5e-4, weight_decay: float=1e-4):
        super().__init__()
        
        self.lr = lr
        self.weight_decay = weight_decay
        
        self.fc1 = nn.Linear(
            in_features=input_size,
            out_features=layer_1
        )

        self.fc2 = nn.Linear(
            in_features=layer_1,
            out_features=layer_2
        )
        
        self.fc3 = nn.Linear(
            in_features=layer_2,
            out_features=layer_3
        )
        
        self.fc4 = nn.Linear(
            in_features=layer_3,
            out_features=layer_4
        )

        self.fc_out = nn.Linear(
            in_features=layer_4,
            out_features=14  # Number of regions  # 143 countries
        )

        self.dropout = nn.Dropout(dropout)
        self.batchnorm1 = nn.BatchNorm1d(layer_1)
        self.batchnorm2 = nn.BatchNorm1d(layer_2)
        self.batchnorm3 = nn.BatchNorm1d(layer_3)
        self.batchnorm4 = nn.BatchNorm1d(layer_4)

    def forward(self, x):
        z = self.fc1(x)
        z = F.relu(z)
        z = self.batchnorm1(z)
        z = self.dropout(z)

        z = self.fc2(z)
        z = F.relu(z)
        z = self.batchnorm2(z)
        z = self.dropout(z)
        
        z = self.fc3(z)
        z = F.relu(z)
        z = self.batchnorm3(z)
        z = self.dropout(z)
        
        z = self.fc4(z)
        z = F.relu(z)
        z = self.batchnorm4(z)
        z = self.dropout(z)

        out = self.fc_out(z)

        return out
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        return optimizer
    

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = accuracy(logits, y)
        self.log('train_loss', loss, on_epoch=True)
        self.log('train_acc', acc, on_epoch=True)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = accuracy(logits, y)
        self.log('val_loss', loss, on_epoch=True)
        self.log('val_acc', acc, on_epoch=True)
        return loss

### Training using PyTorch Lightning

In [13]:
gpu = 1 if torch.cuda.is_available() else 0

trainer = pl.Trainer(
    # logger=True,
    # log_every_n_steps=5,
    gpus=gpu, 
    max_epochs=15
)

hyperparameters = dict(n_layers=4)
    
trainer.logger.log_hyperparams(hyperparameters)

model = ffnn(input_size=4000)
trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type        | Params
-------------------------------------------
0 | fc1        | Linear      | 8.0 M 
1 | fc2        | Linear      | 3.0 M 
2 | fc3        | Linear      | 1.5 M 
3 | fc4        | Linear      | 500 K 
4 | fc_out     | Linear      | 71.6 K
5 | dropout    | Dropout     | 0     
6 | batchnorm1 | BatchNorm1d | 4.0 K 
7 | batchnorm2 | BatchNorm1d | 3.0 K 
8 | batchnorm3 | BatchNorm1d | 2.0 K 
9 | batchnorm4 | BatchNorm1d | 1.0 K 
-------------------------------------------
13.1 M    Trainable params
0         Non-trainable params
13.1 M    Total params
52.347    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [14]:
trainer.callback_metrics

{'train_loss': tensor(0.),
 'train_loss_step': tensor(0.),
 'train_acc': tensor(0.),
 'train_acc_step': tensor(0.),
 'val_loss': tensor(0.),
 'val_acc': tensor(0.),
 'train_loss_epoch': tensor(0.),
 'train_acc_epoch': tensor(0.)}

## What I found:

After trial and error:
- 4 layers is optimal compared to 3 and 5
- At around 8 epochs the validation error starts to increase
- Batchnorm gives a higher accuracy

## Optuna

Trial with only:
- Batch size
- Input size
- Dropout

Best trial:

	 Value: 0.7513267397880554
	 Params:
	 	 batch_size: 45
	 	 input_size: 296
	 	 dropout: 0.1760540639257987


In [12]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.loggers import WandbLogger

def objective(trial):
    # Hyperparameters that is optimized
    batch_size = trial.suggest_int('batch_size', 12, 64)
    
    # Model inputs
    input_size = trial.suggest_int('input_size', 100, 4000)
    layer_1 = trial.suggest_int('layer_1', 500, 2000)
    layer_2 = trial.suggest_int('layer_2', 500, 2000)
    layer_3 = trial.suggest_int('layer_3', 500, 2000)
    layer_4 = trial.suggest_int('layer_4', 500, 2000)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    
    # Optimizer values
    lr = trial.suggest_float('lr', 1e-5, 1e-2)
    weight_decay = trial.suggest_float('weight_decay', 1e-7, 10-2)
    
    x_trial = x[:, :input_size]
    train_loader, val_loader = create_dataloaders(x_trial, y, batch_size=batch_size, train_shuffle=True)
    
    model = ffnn(input_size=input_size, layer_1=layer_1, layer_2=layer_2, layer_3=layer_3, layer_4=layer_4,
                 dropout=dropout, lr=lr, weight_decay=weight_decay)
    
    gpu = 1 if torch.cuda.is_available() else 0
    trainer = pl.Trainer(
        logger=True,
        max_epochs=10,
        # log_every_n_steps=5,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor='val/acc')],
        gpus=gpu,
    )
    
    hyperparameters = dict(batch_size=batch_size,
                           input_size=input_size,
                           layer_1=layer_1,
                           layer_2=layer_2,
                           layer_3=layer_3,
                           layer_4=layer_4,
                           dropout=dropout, 
                           lr=lr,
                           weight_decay=weight_decay)
    
    trainer.logger.log_hyperparams(hyperparameters)
    
    trainer.fit(model, train_loader, val_loader)
    
    return trainer.callback_metrics['val/acc'].item()

In [13]:
print('Finding optimal hyperparameters using Optuna...')
pruner = optuna.pruners.MedianPruner(n_startup_trials=5)    
    
study = optuna.create_study(direction='maximize', pruner=pruner)
study.optimize(objective, n_trials=50)
    
print('Best trial:')
trial = study.best_trial

clear_output()

print(f'\t Value: {trial.value}')
print(f'\t Params:')
for key, value in trial.params.items():
    print(f'\t \t {key}: {value}')

[I 2021-11-22 21:32:33,047] A new study created in memory with name: no-name-3722b8e2-6d0e-44bf-9d64-0d623516c11c
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding optimal hyperparameters using Optuna...



  | Name       | Type        | Params
-------------------------------------------
0 | fc1        | Linear      | 2.2 M 
1 | fc2        | Linear      | 1.1 M 
2 | fc3        | Linear      | 905 K 
3 | fc4        | Linear      | 816 K 
4 | fc_out     | Linear      | 153 K 
5 | dropout    | Dropout     | 0     
6 | batchnorm1 | BatchNorm1d | 1.9 K 
7 | batchnorm2 | BatchNorm1d | 2.4 K 
8 | batchnorm3 | BatchNorm1d | 1.5 K 
9 | batchnorm4 | BatchNorm1d | 2.1 K 
-------------------------------------------
5.2 M     Trainable params
0         Non-trainable params
5.2 M     Total params
20.904    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation sanity check: 0it [00:00, ?it/s]

d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

[W 2021-11-22 21:36:41,927] Trial 2 failed because of the following error: RuntimeError('CUDA error: an illegal memory access was encountered\nCUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.\nFor debugging consider passing CUDA_LAUNCH_BLOCKING=1.')
Traceback (most recent call last):
  File "d:\github\02456-deep-learning\venv\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\spetr\AppData\Local\Temp/ipykernel_17344/1926304977.py", line 48, in objective
    trainer.fit(model, train_loader, val_loader)
  File "d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 735, in fit
    self._call_and_handle_interrupt(
  File "d:\github\02456-deep-learning\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 682, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "d:\github\02

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.